In [1]:
!pip install einops
import torch.nn as nn
import torch.nn.functional as F
from contextlib import contextmanager
import torchvision
import torch
from torchvision.datasets import MNIST,CIFAR10
import matplotlib.pyplot as plt
from six import add_metaclass
from torch.nn import init
import torch.optim as optim
import os
from torch.utils.data import DataLoader,random_split
from torchvision.utils import save_image
import numpy as np
from PIL import Image
from torchvision import datasets, transforms
from contextlib import contextmanager
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop
from six import add_metaclass
from torch.optim import Adam, lr_scheduler

from kornia.augmentation import RandomCrop, Normalize
from argparse import ArgumentParser
import errno
from einops import rearrange

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
def weights_init(m):
    # Code taken from https://discuss.pytorch.org/t/initializing-rnn-gru-and-lstm-correctly/23605/8
    parameters = m.state_dict()
    for each_key in parameters.keys():
        print(f'Init-{each_key}')
        if 'weight_ih' in each_key:
            nn.init.orthogonal_(parameters[each_key])
        elif 'weight_hh' in each_key:
            nn.init.orthogonal_(parameters[each_key])
        elif 'bias' in each_key:
            nn.init.constant_(parameters[each_key], val=0)

In [3]:
class ReNet(nn.Module):
    def __init__(self, input_size, hidden_size, kernel_size=(4, 4), rnn='LSTM', depth=(1,1)):
        super(ReNet, self).__init__()
        if rnn == 'GRU':
            rnn = nn.GRU
        elif rnn == 'LSTM':
            rnn = nn.LSTM
        
        self.lstm_h = rnn(input_size, hidden_size, bias=False, num_layers=depth[0], bidirectional=True)
        self.lstm_v = rnn(hidden_size * 2, hidden_size, bias=False, num_layers=depth[1], bidirectional=True)

        if isinstance(kernel_size, int):
            self.kernel_size = (kernel_size, kernel_size)
        else:
            self.kernel_size = kernel_size
        
        self.lstm_h.apply(weights_init)
        self.lstm_v.apply(weights_init)

    def forward(self, x):
        k_w, k_h = self.kernel_size 
        b, c, h, w = x.size()
        
        assert h % k_h == 0 and w % k_w == 0, 'input size does not match with kernel size'
        x = rearrange(x, 'b c (h1 h2) (w1 w2) -> h1 (b w1) (c h2 w2)', w2=k_w, h2=k_h)
        x, _ = self.lstm_h(x)
        x = rearrange(x, 'h1 (b w1) (c h2 w2) -> w1 (b h1) (c h2 w2)', b=b, w2=k_w, h2=k_h)
        x, _ = self.lstm_v(x)
        x = rearrange(x, 'w1 (b h1) (c h2 w2) -> b (c h2 w2) h1 w1', b=b, w2=k_w, h2=k_h)
        return x
        

In [4]:
renet = nn.Sequential(
    ReNet(4, 128, kernel_size=(2,2)), 
    ReNet(1024,128,kernel_size=(2,2)),
    nn.Flatten(),
    nn.Linear(256 * 7 * 7, 4096),
    nn.ReLU(),
    nn.Linear(4096, 10),
)


device = torch.device('cuda:0')
renet = renet.to(device)

Init-weight_ih_l0
Init-weight_hh_l0
Init-weight_ih_l0_reverse
Init-weight_hh_l0_reverse
Init-weight_ih_l0
Init-weight_hh_l0
Init-weight_ih_l0_reverse
Init-weight_hh_l0_reverse
Init-weight_ih_l0
Init-weight_hh_l0
Init-weight_ih_l0_reverse
Init-weight_hh_l0_reverse
Init-weight_ih_l0
Init-weight_hh_l0
Init-weight_ih_l0_reverse
Init-weight_hh_l0_reverse


In [5]:

transform_list = [
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]
'''
torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                            transforms.CenterCrop((448,448)),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
])
'''
mnist_train_data = datasets.MNIST(root='data', train=True,download=True, transform=transforms.Compose(transform_list))
mnist_test_data = datasets.MNIST(root='data', train=False,download=True, transform=transforms.Compose(transform_list))

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
mnist_train_loader = DataLoader(mnist_train_data,shuffle=True,batch_size=1024,pin_memory=True)
mnist_test_loader = DataLoader(mnist_test_data,shuffle=True,batch_size=1024,pin_memory=True)

In [7]:
num_epochs = 5
learning_rate = 0.01

optimizer = torch.optim.SGD(renet.parameters(), lr=learning_rate)  
criterion = nn.CrossEntropyLoss()

In [8]:
for epoch in range(num_epochs):
    for images,labels in mnist_train_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = renet(images)

        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    correct = 0
    total = 0
    accuracy = 0
    for images, labels in mnist_test_loader:
        images, labels = images.to(device), labels.to(device)
            
        outputs = renet(images)
            
        predictions = torch.argmax(outputs, dim=1)
        correct += (predictions == labels).sum()
            
        total += len(labels)
            
        accuracy = correct * 100 // total
    print("Epoch: {}, Test Accuracy: {}%".format(epoch, accuracy))





Epoch: 0, Test Accuracy: 10%
Epoch: 1, Test Accuracy: 10%
Epoch: 2, Test Accuracy: 11%
Epoch: 3, Test Accuracy: 11%
Epoch: 4, Test Accuracy: 11%


KeyboardInterrupt: 